In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

/Users/vadim/anaconda3/envs/labs/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
MODEL_NAME = "IlyaGusev/saiga2_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = ("Ты - Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. Отвечай на вопросы пользователя с положительным эмоциональным окрасом. Пользователь: Как зовут главного персонажа игры Мортал Комбат? Сайга: Конечно же это Лю Кан! Как же я люблю эту замечательную игру!!! Пользователь: Какая самая высокая гора в России? Сайга: Самая высокая гора России - Эльбрус, я слышал, что оттуда восхитительный вид!")

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype=torch.float16,
    offload_folder="offload",
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    offload_folder="offload"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

print(generation_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 3584,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.5,
  "top_k": 30,
  "top_p": 0.9
}


In [3]:
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]
        
    def add_system_prompt(self, new_prompt):
        if self.messages and self.messages[0]["role"] == "system":
            self.messages[0]["content"] = new_prompt
    
    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [4]:
data = []
with open('questions.txt', 'r') as file:
    for line in file:
        data.append(line.strip())
data

['Сколько планет в Солнечной системе?',
 'Какая самая высокая гора в мире?',
 'Какая самая глубокая точка в Мировом океане?',
 'Кто написал "Войну и мир"?',
 'Какова длина Великой Китайской стены?',
 'Кто изобрел лампочку?',
 'Какой самый большой океан?',
 'Где находится Мачу-Пикчу?',
 'Кто такой Альберт Эйнштейн?',
 'Какая самая большая страна в мире?',
 'Какая самая маленькая страна в мире?',
 'Кто написал "Гамлета"?',
 'Какой самый длинный река в мире?',
 'Какая самая горячая планета в Солнечной системе?',
 'Какой самый большой материк?',
 'Кто написал "Преступление и наказание"?',
 'Какой газ составляет большую часть атмосферы Земли?',
 'Какой элемент имеет символ Au?',
 'Кто был первым человеком в космосе?',
 'Как называется столица Японии?',
 'Какой самый маленький океан?',
 'Кто написал "Одиссею"?',
 'Какая страна производит больше всего кофе?',
 'Какой самый высокий водопад в мире?',
 'Кто был первым президентом США?',
 'Какая планета ближе всего к Солнцу?',
 'Как называется са

In [5]:
with open("results/results_instruct.txt", 'w', encoding='utf-8') as file:
    for i in range(0, len(data)):
        conversation = Conversation()
        conversation.add_user_message(data[i])
        prompt = conversation.get_prompt(tokenizer)
        print(data[i])
        output = generate(model, tokenizer, prompt, generation_config)
        print(output)
        file.write(f"Вопрос: {data[i]}\nОтвет: {output}\n\n")
        print("\n==============================\n")

Сколько планет в Солнечной системе?
В настоящее время известно 8 планет Солнечной системы (Меркурий, Венера, Земля, Марс, Юпитер, Сатурн, Уран и Нептун). Однако есть предположения об существовании еще нескольких неизвестных объектов внутри пояса астероидов или за пределами орбиты Нептуна.


Какая самая высокая гора в мире?
Самой высокой горой в мире является Хималаи, расположенная между Непалом и Тибетом. Она имеет несколько вершин, самым известным из которых является Монблан (8 848 м), но есть еще несколько более высоких горных пиков, таких как Дхаулагири (8 163 м) или Аннуппурна (8 091 м).


Какая самая глубокая точка в Мировом океане?
Самой глубокой точкой Мирового океана является Черное море (подводная горловина). Оно находится у берегов Азорских островов, имеет максимальные глубины около 10 тысяч метров. Это одна из самых глубоких точек во вселенной.


Кто написал "Войну и мир"?
Алексей Николаевич Толстой.


Какова длина Великой Китайской стены?
Великая китайская стена имеет длину